In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import statistics
import os
import shutil

In [ ]:
# Getting the Dataset using the Kaggle API
!kaggle datasets download rashikrahmanpritom/data-science-job-posting-on-glassdoor

# Unziping the Data
shutil.unpack_archive("data-science-job-posting-on-glassdoor.zip",extract_dir='data')

# Deleting the Zip File and the Cleaned Version
os.remove("data-science-job-posting-on-glassdoor.zip")
os.remove("data/Cleaned_DS_Jobs.csv")

# Renaming our dataset
os.rename("data/Uncleaned_DS_jobs.csv", "data/Data_Science_Jobs.csv")

In [ ]:
# Starting the dataframe and correcting the index
df = pd.read_csv("data/Data_Science_Jobs.csv",index_col='index')

In [ ]:
# Getting the Salary Estimate
def get_salary_estimate(x):
    estimate = ''
    for char in x:
        if char.isdigit() or char == '-':
            estimate += char 
    return estimate

df['Salary Estimate'] = df['Salary Estimate'].apply(get_salary_estimate)

In [ ]:
# Cleaning the Company Name
def get_company_name(x):
    if str("\n") in x:
        array = x.split(str("\n"))
        return array[0].strip()
    else:
        return x
    
df['Company Name'] = df['Company Name'].apply(get_company_name)

In [ ]:
# Cleaning the Company Size
def get_size(x):
    size = ''
    x = x.replace(' to ','-')
    for char in x:
        if char.isdigit() or char == '-':
            size += char 
    return size
    
df['Size'] = df['Size'].apply(get_size)

In [ ]:
# Getting the Minimum Salary
df['Min Salary'] = df['Salary Estimate'].apply(lambda x: int(min(x.split('-'))))

# Getting the Maximum Salary
df['Max Salary'] = df['Salary Estimate'].apply(lambda x: int(max(x.split('-'))))

# Getting the Average Salary
df['Avg Salary'] = df['Salary Estimate'].apply(lambda x: statistics.mean(list(map(int,x.split('-')))))

In [ ]:
# Creating the Job State Column
def get_job_state(x):
    # States that does not have the acronym
    standardize_states = {"Utah":"UT",
                      "United States":"US",
                      "Remote":"RMT",
                      "New Jersey":"NJ",
                      "Texas":"TX",
                      "California":"CA"}
    
    try:
        state = x.split(",")[1].strip()
    except:
        for state_key,acronym in standardize_states.items():
            if x == state_key:
                state = acronym
    return state

find_different = df['Location'].apply(get_job_state)
df['Job State'] = df['Location'].apply(get_job_state)

In [ ]:
# Creating the Same State Boolean Column
def get_headquarters_state(x):
    try:
        state = x.split(",")[1].strip()
    except:
        state = x
    return state

headquarters_states = df['Headquarters'].apply(get_headquarters_state)
df['Same State'] = df['Job State'] == headquarters_states

In [ ]:
# Including the most appeared skills in boolean columns form using Lambda
df['Python - Required'] = df['Job Description'].apply(lambda x: True if "python" in x.lower() else False)
df['Excel - Required'] = df['Job Description'].apply(lambda x: True if "excel" in x.lower() else False)
df['Hadoop - Required'] = df['Job Description'].apply(lambda x: True if "hadoop" in x.lower() else False)
df['Spark - Required'] = df['Job Description'].apply(lambda x: True if "spark" in x.lower() else False)
df['AWS - Required'] = df['Job Description'].apply(lambda x: True if "aws" in x.lower() else False)
df['Tableau - Required'] = df['Job Description'].apply(lambda x: True if "tableau" in x.lower() else False)
df['Big Data - Required'] = df['Job Description'].apply(lambda x: True if "big data" in x.lower() else False)


In [ ]:
# Getting a simple version of the Job
def get_simpler_job(x):
    x = x.lower()
    if "data scientist" in x:
        simpler_job = "Data Scientist"
    elif "machine learning engineer" in x:
        simpler_job = "M.L.E"
    elif "analyst" in x:
        simpler_job = "Analyst"
    elif "data engineer" in x:
        simpler_job = "Data Engineer"
    elif "computer scientist" in x:
        simpler_job = "Computer Scientist"
    else:
        simpler_job = np.nan
    return simpler_job
    
df['Simpler Job'] = df['Job Title'].apply(get_simpler_job)

In [ ]:
# Ending the project getting the desired level of the job
def get_job_level(x):
    x = x.lower()
    if "jr" in x:
        level = "Junior"
    elif any(k in x for k in ["principal","senior",'experienced','sr']):
        level = "Senior"
    else:
        level = np.nan
    return level

df['Desired Level'] = df['Job Title'].apply(get_job_level)

In [ ]:
# Calculating the company age
df['Company Age'] = df['Founded'].apply(lambda x: datetime.now().year-x if x>-1 else x)


In [ ]:
# It looks better than before :/
df